# Intialisation Steps
Connecting to Snowflake

## Importing required modules

In [1]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

## Snowflake connection parameter

In [2]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# ICD grouper table

In [3]:
%%read_sql

select
    *
from
    RWD_DB.RWD_REFERENCE_LIBRARY.ICD_GROUPER
where
    --left(level_4, 4) in ('G122') -- motor neurone disease
     (level_4_long_description_icd10 ilike '%macular%'
        and level_4_long_description_icd10 ilike '%edema%')

Query started at 05:56:40 PM India Standard TimeInitiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 0.49 m

,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,3727,E00 - E8989,Endocrine nutritional and metabolic diseases,E08-E13,Diabetes mellitus,E08-E089,Diabetes mellitus due to underlying condition,E08311,24950,Secondary diabetes mellitus with ophthalmic ma...,Diab due to undrl cond w unsp diabetic rtnop w...,Diabetes mellitus due to underlying condition ...,2017-11-02,2017-11-02
1,3728,E00 - E8989,Endocrine nutritional and metabolic diseases,E08-E13,Diabetes mellitus,E08-E089,Diabetes mellitus due to underlying condition,E08319,24950,Secondary diabetes mellitus with ophthalmic ma...,Diab due to undrl cond w unsp diab rtnop w/o m...,Diabetes mellitus due to underlying condition ...,2017-11-02,2017-11-02
2,3730,E00 - E8989,Endocrine nutritional and metabolic diseases,E08-E13,Diabetes mellitus,E08-E089,Diabetes mellitus due to underlying condition,E08321,No map,No map,Diab d/t undrl cond w mild nonprlf diab rtnop ...,Diabetes mellitus due to underlying condition ...,2017-11-02,2017-11-02
3,3731,E00 - E8989,Endocrine nutritional and metabolic diseases,E08-E13,Diabetes mellitus,E08-E089,Diabetes mellitus due to underlying condition,E083211,No map,No map,Diabetes with mild nonp rtnop with macular ede...,Diabetes mellitus due to underlying condition ...,2017-11-02,2017-11-02
4,3732,E00 - E8989,Endocrine nutritional and metabolic diseases,E08-E13,Diabetes mellitus,E08-E089,Diabetes mellitus due to underlying condition,E083212,No map,No map,Diabetes with mild nonp rtnop with macular ede...,Diabetes mellitus due to underlying condition ...,2017-11-02,2017-11-02
5,3733,E00 - E8989,Endocrine nutritional and metabolic diseases,E08-E13,Diabetes mellitus,E08-E089,Diabetes mellitus due to underlying condition,E083213,No map,No map,Diabetes with mild nonp rtnop with macular ede...,Diabetes mellitus due to underlying condition ...,2017-11-02,2017-11-02
6,3734,E00 - E8989,Endocrine nutritional and metabolic diseases,E08-E13,Diabetes mellitus,E08-E089,Diabetes mellitus due to underlying condition,E083219,No map,No map,Diabetes with mild nonp rtnop with macular ede...,Diabetes mellitus due to underlying condition ...,2017-11-02,2017-11-02
7,3735,E00 - E8989,Endocrine nutritional and metabolic diseases,E08-E13,Diabetes mellitus,E08-E089,Diabetes mellitus due to underlying condition,E08329,No map,No map,Diab d/t undrl cond w mild nonprlf diab rtnop ...,Diabetes mellitus due to underlying condition ...,2017-11-02,2017-11-02
8,3736,E00 - E8989,Endocrine nutritional and metabolic diseases,E08-E13,Diabetes mellitus,E08-E089,Diabetes mellitus due to underlying condition,E083291,No map,No map,Diabetes with mild nonp rtnop without macular ...,Diabetes mellitus due to underlying condition ...,2017-11-02,2017-11-02
9,3737,E00 - E8989,Endocrine nutritional and metabolic diseases,E08-E13,Diabetes mellitus,E08-E089,Diabetes mellitus due to underlying condition,E083292,No map,No map,Diab with mild nonp rtnop without macular edem...,Diabetes mellitus due to underlying condition ...,2017-11-02,2017-11-02


## Swarali notes:
- Using level_4_short_description_icd10 variable because description includes retinal vein occlusion and macular edema

In [23]:
%%read_sql

create or replace temporary table st_rvo_me_ref1 as

select
    'dx' as cat1,
    case
        when left(level_4, 4) in ('H348')
        then 'rvo'
        else 'other_ME'
        end as cat2,
    null as cat3,
    null as cat4,
    null as cat5,
    null as cat6,
    'icd_10' as type,
    level_4 as value,
    level_4_long_description_icd10 as description,
    current_date() as active_date,
    'Nish' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    'RWD_DB.RWD.ICD_GROUPER' as source
from 
    RWD_DB.RWD.ICD_GROUPER
where
    left(level_4, 4) in ('H348') -- retinal vein occlusion
    or (level_4_short_description_icd10 ilike '%rtnl%'
        and level_4_short_description_icd10 ilike '%vein%'
        and level_4_short_description_icd10 ilike '%occlusion%'
        and level_4_short_description_icd10 ilike '%macular%'
        and level_4_short_description_icd10 ilike '%edema%')

Query started at 12:33:33 PM GMT Daylight Time; Query executed in 0.06 m

,status
0,Table NK_MND_REF1 successfully created.


In [27]:
%%read_sql

create or replace temporary table st_rvo_me_ref2 as

select
    'dx' as cat1,
    case
        when left(level_4, 4) in ('H348')
        then 'rvo'
        else 'other_ME'
        end as cat2,
    null as cat3,
    null as cat4,
    null as cat5,
    null as cat6,
    'icd_9' as type,
    icd9_mapped_codes as value,
    icd9_description as description,
    current_date() as active_date,
    'Nish' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    'RWD_DB.RWD.ICD_GROUPER' as source
from 
    RWD_DB.RWD.ICD_GROUPER
where
    ((left(level_4, 4) in ('H348') -- retinal vein occlusion
    or  level_4_short_description_icd10 ilike '%rtnl%'
        and level_4_short_description_icd10 ilike '%vein%'
        and level_4_short_description_icd10 ilike '%occlusion%'
        and level_4_short_description_icd10 ilike '%macular%'
        and level_4_short_description_icd10 ilike '%edema%'))
    and icd9_mapped_codes != 'No map'

Query started at 12:35:16 PM GMT Daylight Time; Query executed in 0.05 m

,status
0,Table NK_MND_REF2 successfully created.


In [29]:
%%read_sql

create or replace table st_rvo_me_ref as

select * from st_rvo_me_ref1
union
select * from st_rvo_me_ref2
order by type, cat2

Query started at 12:35:45 PM GMT Daylight Time; Query executed in 0.06 m

,status
0,Table NK_MND_REF successfully created.


## Upload into snowflake

In [ ]:
rvo_ref = pd.read_excel("in/ref.xlsx")

In [ ]:
snow.drop_table("st_rvo_me_ref")

In [ ]:
snow.upload_dataframe(als_ref, "st_rvo_me_ref")

In [5]:
%%read_sql

select *
from st_rvo_me_ref
where deactive_date is null

Query started at 05:24:20 PM GMT Daylight Time; Query executed in 0.08 m

,cat1,cat2,cat3,cat4,cat5,cat6,type,value,description,active_date,active_reason,deactive_date,deactive_reason,source
0,dx,ftd,None,None,None,None,icd_10,G310,Frontotemporal dementia,2018-07-16,Nish,None,None,RWD_DB.RWD.ICD_GROUPER
1,dx,ftd,None,None,None,None,icd_10,G3109,Other frontotemporal dementia,2018-07-16,Nish,None,None,RWD_DB.RWD.ICD_GROUPER
2,dx,mnd,None,None,None,None,icd_10,G122,Motor neuron disease,2018-07-16,Nish,None,None,RWD_DB.RWD.ICD_GROUPER
3,dx,mnd,None,None,None,None,icd_10,G1220,Motor neuron disease unspecified,2018-07-16,Nish,None,None,RWD_DB.RWD.ICD_GROUPER
4,dx,mnd,None,None,None,None,icd_10,G1221,Amyotrophic lateral sclerosis,2018-07-16,Nish,None,None,RWD_DB.RWD.ICD_GROUPER
5,dx,mnd,None,None,None,None,icd_10,G1222,Progressive bulbar palsy,2018-07-16,Nish,None,None,RWD_DB.RWD.ICD_GROUPER
6,dx,mnd,None,None,None,None,icd_10,G1223,Primary lateral sclerosis,2018-07-16,Nish,None,None,RWD_DB.RWD.ICD_GROUPER
7,dx,mnd,None,None,None,None,icd_10,G1224,Familial motor neuron disease,2018-07-16,Nish,None,None,RWD_DB.RWD.ICD_GROUPER
8,dx,mnd,None,None,None,None,icd_10,G1229,Other motor neuron disease,2018-07-16,Nish,None,None,RWD_DB.RWD.ICD_GROUPER
9,dx,mnd,None,None,None,None,icd_10,G1225,Progressive spinal muscle atrophy,2018-07-16,Nish,None,None,RWD_DB.RWD.ICD_GROUPER


# Analysis plan

<img src="in/cohort.png", width=700>

# Cohort in obs. window

<img src="in/cohort.png", width=700>

In [6]:
%%read_sql

create or replace table st_rvo_obs_cohort as

select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    diagnosis in (select distinct value from st_rvo_me_ref where cat2 = 'rvo' and deactive_date is null)
    and year_of_service between '2015-07-01' and '2017-06-30'

Query started at 05:24:35 PM GMT Daylight Time; Query executed in 10.08 m

,status
0,Table NK_MND_OBS_COHORT successfully created.


In [7]:
snow.select("select count(distinct patient_id) from st_rvo_obs_cohort where left(patient_id, 5) != 'XXX -'")

,COUNT(DISTINCT PATIENT_ID)
0,36262


## Index date

In [8]:
%%read_sql

create or replace table st_rvo_index as

select
    patient_id,
    min(year_of_service) as index_date
from
    st_rvo_obs_cohort
where
    left(patient_id, 5) != 'XXX -'
group by 1

Query started at 05:34:41 PM GMT Daylight Time; Query executed in 0.06 m

,status
0,Table NK_MND_INDEX successfully created.


In [9]:
snow.select("select * from st_rvo_index order by index_date desc limit 5")

,patient_id,index_date
0,lDDQBoRr4MRLoqpU,2017-06-30
1,1aQrFEq2gy5GBI+V,2017-06-30
2,d1yFZGiaC2P5i29J,2017-06-30
3,g2ndO4guv59oxZG+,2017-06-30
4,rAJjpT6Pe/SWD+jr,2017-06-30


In [10]:
snow.select("select count(patient_id) from st_rvo_index where left(patient_id, 5) = 'XXX -'")

,COUNT(PATIENT_ID)
0,0


# Continuous coverage

## Cont. cov from medical claims
<img src="in/cohort.png", width=700>

### Extract submits header

In [11]:
%%read_sql

create or replace table st_rvo_cont_med as

select
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) as patient_id,
    claim_number,
    claim_type_code,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_HEADER
where
    left(encrypted_key_1, 8)||left(encrypted_key_2, 8) in (select patient_id from st_rvo_index)
    and year_of_service between '2014-07-01' and '2018-06-30'

Query started at 05:34:47 PM GMT Daylight Time; Query executed in 5.89 m

,status
0,Table NK_MND_CONT_MED successfully created.


### Check for duplicates

In [12]:
%%read_sql

select count(*) from st_rvo_cont_med

Query started at 05:40:40 PM GMT Daylight Time; Query executed in 0.04 m

,COUNT(*)
0,2715107


In [13]:
%%read_sql

with t1 as (select distinct * from st_rvo_cont_med)
select count(*) from t1

Query started at 05:40:43 PM GMT Daylight Time; Query executed in 0.09 m

,COUNT(*)
0,2700266


In [14]:
%%read_sql

drop table if exists st_rvo_cont_med_unique;
create or replace table st_rvo_cont_med_unique as
select distinct * from st_rvo_cont_med;
drop table st_rvo_cont_med;

Query started at 05:40:48 PM GMT Daylight Time; Query executed in 0.05 mQuery started at 05:40:51 PM GMT Daylight Time; Query executed in 0.06 mQuery started at 05:40:55 PM GMT Daylight Time; Query executed in 0.04 m

,status
0,NK_MND_CONT_MED successfully dropped.


### Summarise medical claims

In [15]:
%%read_sql

create or replace table st_rvo_cont_med_tally as

select
    a.patient_id,
    sum(case
           when datediff(d, a.index_date, b.year_of_service) between -365 and -1
           then 1
           else 0
           end) as claim_tally_pre,
    sum(case
           when datediff(d, a.index_date, b.year_of_service) between 0 and 365
           then 1
           else 0
           end) as claim_tally_post
from
    st_rvo_index a
        join st_rvo_cont_med_unique b
            on a.patient_id = b.patient_id
group by 1

Query started at 05:40:57 PM GMT Daylight Time; Query executed in 0.07 m

,status
0,Table NK_MND_CONT_MED_TALLY successfully created.


### Final counts

In [16]:
a = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally")
b = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally where claim_tally_pre > 0 and claim_tally_post > 0")
c = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally where claim_tally_pre > 1 and claim_tally_post > 1")
d = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally where claim_tally_pre > 2 and claim_tally_post > 2")
e = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally where claim_tally_pre > 3 and claim_tally_post > 3")
f = snow.select("select count(distinct patient_id) from st_rvo_cont_med_tally where claim_tally_pre > 4 and claim_tally_post > 4")

lst = ['pts_obs_window',
       '1_claim_pre_and_post',
       '2_claims_pre_and_post',
       '3_claims_pre_and_post',
       '4_claims_pre_and_post',
       '5_claims_pre_and_post',]

pd.concat([a,b,c,d,e,f], keys=lst)

,,COUNT(DISTINCT PATIENT_ID)
pts_obs_window,0,36262
1_claim_pre_and_post,0,33110
2_claims_pre_and_post,0,30707
3_claims_pre_and_post,0,28269
4_claims_pre_and_post,0,26055
5_claims_pre_and_post,0,24080


## Final cohort

In [17]:
%%read_sql

create or replace table st_rvo_cohort as

select
    patient_id,
    index_date
from 
    st_rvo_index
where
    patient_id in (select patient_id 
                   from st_rvo_cont_med_tally 
                   where claim_tally_pre > 1
                   and claim_tally_post > 1)

Query started at 05:41:13 PM GMT Daylight Time; Query executed in 0.08 m

,status
0,Table NK_MND_COHORT successfully created.


In [18]:
snow.select("select count(distinct patient_id) from st_rvo_cohort")

,COUNT(DISTINCT PATIENT_ID)
0,30707


# Raw data extracts

Pull all diagnoses data for the cohort of patients from July 1st 2014, to June 31st 2018
<img src="in/cohort.png", width=700>

## Claim header

In [19]:
%%read_sql

create or replace table st_rvo_header_all as 

select
     'XXXXXXXXXXXXXXXX' as patient_id,
     claim_number,
     received_date,
     claim_type_code,
     statement_from,
     statement_to,
     min_service_from,
     max_service_to,
     total_charge,
     total_allowed,
     drg_code,
     type_bill,
     admission_date,
     admit_type_code,
     admit_src_code,
     discharge_hour,
     discharge_status,
     new_medicare_source,
     year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_HEADER
where
     claim_number in (select distinct claim_number 
                      from st_rvo_cont_med_unique
                      where patient_id in (select patient_id
                                           from st_rvo_cohort))

Query started at 05:41:21 PM GMT Daylight Time; Query executed in 3.09 m

,status
0,Table NK_MND_HEADER_ALL successfully created.


In [20]:
%%read_sql

 begin;
 update st_rvo_header_all a
    set a.patient_id = b.patient_id
   from st_rvo_cont_med_unique b
  where a.claim_number = b.claim_number;
commit;

Query started at 05:44:27 PM GMT Daylight Time; Query executed in 0.03 mQuery started at 05:44:29 PM GMT Daylight Time; Query executed in 0.13 mQuery started at 05:44:37 PM GMT Daylight Time; Query executed in 0.04 m

,status
0,Statement executed successfully.


In [21]:
%%read_sql

select
    count(patient_id),
    count(distinct patient_id),
    sum(case
           when patient_id = 'XXXXXXXXXXXXXXXX'
           then 1
           else 0
           end) as null_pt_id,
    sum(case
           when left(patient_id, 5) = 'XXX -'
           then 1
           else 0
           end) as faulty_pt_id
from st_rvo_header_all

Query started at 05:44:39 PM GMT Daylight Time; Query executed in 0.04 m

,COUNT(PATIENT_ID),COUNT(DISTINCT PATIENT_ID),null_pt_id,faulty_pt_id
0,2604522,30707,0,0


## Diagnoses

In [22]:
%%read_sql

create or replace table st_rvo_dx_all as

select
    'XXXXXXXXXXXXXXXX' as patient_id,
    claim_number,
    diagnosis,
    diagnosis_sequence,
    statement_from,
    statement_to,
    min_service_from,
    max_service_to,
    received_date,
    year_of_service
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_DIAGNOSIS
where
    claim_number in (select distinct claim_number
                     from st_rvo_header_all)

Query started at 05:44:41 PM GMT Daylight Time; Query executed in 4.80 m

,status
0,Table NK_MND_DX_ALL successfully created.


In [23]:
%%read_sql

begin;
update st_rvo_dx_all a
set a.patient_id = b.patient_id
from st_rvo_header_all b
where a.claim_number = b.claim_number

Query started at 05:49:30 PM GMT Daylight Time; Query executed in 0.03 mQuery started at 05:49:31 PM GMT Daylight Time; Query executed in 0.15 m

,number of rows updated,number of multi-joined rows updated
0,10004385,127905


In [24]:
%%read_sql

select
    count(patient_id),
    count(distinct patient_id),
    sum(case
           when patient_id = 'XXXXXXXXXXXXXXXX'
           then 1
           else 0
           end) as null_pt_id,
    sum(case
           when left(patient_id, 5) = 'XXX -'
           then 1
           else 0
           end) as faulty_pt_id
    
from st_rvo_dx_all

Query started at 05:49:40 PM GMT Daylight Time; Query executed in 0.04 m

,COUNT(PATIENT_ID),COUNT(DISTINCT PATIENT_ID),null_pt_id,faulty_pt_id
0,10004385,30707,0,0


## Patient raw

In [25]:
%%read_sql

create or replace table st_rvo_pat_all as

select
    'XXXXXXXXXXXXXXXX' as patient_id,
    claim_number,
    patient_gender,
    patient_dob
from
    RWD_DB.RWD.RAVEN_CLAIMS_SUBMITS_PATIENT
where
    claim_number in (select distinct claim_number
                     from st_rvo_header_all)

Query started at 05:49:43 PM GMT Daylight Time; Query executed in 3.44 m

,status
0,Table NK_MND_PAT_ALL successfully created.


In [26]:
%%read_sql

 begin;
 update st_rvo_pat_all a
    set a.patient_id = b.patient_id
   from st_rvo_header_all b
  where a.claim_number = b.claim_number;
commit;

Query started at 05:53:09 PM GMT Daylight Time; Query executed in 0.04 mQuery started at 05:53:11 PM GMT Daylight Time; Query executed in 0.13 mQuery started at 05:53:19 PM GMT Daylight Time; Query executed in 0.04 m

,status
0,Statement executed successfully.


In [27]:
%%read_sql

create or replace table st_rvo_pat_all_unique as

select
    patient_id,
    min(patient_dob) as dob,
    min(patient_gender) as gender
from
    st_rvo_pat_all
group by 1

Query started at 05:53:21 PM GMT Daylight Time; Query executed in 0.06 m

,status
0,Table NK_MND_PAT_ALL_UNIQUE successfully created.


In [36]:
%%read_sql

select
    count(patient_id),
    count(distinct patient_id),
    sum(case
           when patient_id = 'XXXXXXXXXXXXXXXX'
           then 1
           else 0
           end) as null_pt_id,
    sum(case
           when left(patient_id, 5) = 'XXX -'
           then 1
           else 0
           end) as faulty_pt_id
    
from st_rvo_pat_all_unique

Query started at 05:54:28 PM GMT Daylight Time; Query executed in 0.04 m

,COUNT(PATIENT_ID),COUNT(DISTINCT PATIENT_ID),null_pt_id,faulty_pt_id
0,30707,30707,0,0


# Master table

## Aggregate comorbidity Dx

In [28]:
snow.select("select distinct cat2 from st_rvo_me_ref")

,cat2
0,ftd
1,mnd


In [29]:
%%read_sql

create or replace table st_rvo_comorbidites_agg as 

select
    a.patient_id,
    1 as rvo,
    max(case
           when b.diagnosis in (select distinct value
                                from st_rvo_me_ref
                                where cat2 = 'other_ME' and deactive_date is null)
           and datediff(d, a.index_date, b.year_of_service) between -365 and 365                   
           then 1
           else 0
           end) as other_ME
from
    st_rvo_cohort a
        join st_rvo_dx_all b
            on a.patient_id = b.patient_id
group by 1

Query started at 05:53:25 PM GMT Daylight Time; Query executed in 0.10 m

,status
0,Table NK_MND_COMORBIDITES_AGG successfully cre...


## Patient age

In [30]:
%%read_sql

create or replace table st_rvo_patient_demographics as 

select
    a.patient_id,
    b.gender,
    datediff(yy, b.dob, a.index_date) as age,
    case
        when datediff(yy, b.dob, a.index_date) between 0 and 4 then '00-04'
        when datediff(yy, b.dob, a.index_date) between 5 and 9 then '05-09'
        when datediff(yy, b.dob, a.index_date) between 10 and 14 then '10-14'
        when datediff(yy, b.dob, a.index_date) between 15 and 19 then '15-19'
        when datediff(yy, b.dob, a.index_date) between 20 and 24 then '20-24'
        when datediff(yy, b.dob, a.index_date) between 25 and 29 then '25-29'
        when datediff(yy, b.dob, a.index_date) between 30 and 34 then '30-34'
        when datediff(yy, b.dob, a.index_date) between 35 and 39 then '35-39'
        when datediff(yy, b.dob, a.index_date) between 40 and 44 then '40-44'
        when datediff(yy, b.dob, a.index_date) between 45 and 49 then '45-49'
        when datediff(yy, b.dob, a.index_date) between 50 and 54 then '50-54'
        when datediff(yy, b.dob, a.index_date) between 55 and 59 then '55-59'
        when datediff(yy, b.dob, a.index_date) between 60 and 64 then '60-64'
        when datediff(yy, b.dob, a.index_date) between 65 and 69 then '65-69'
        when datediff(yy, b.dob, a.index_date) between 70 and 74 then '70-74'
        when datediff(yy, b.dob, a.index_date) between 75 and 79 then '75-79'
        when datediff(yy, b.dob, a.index_date) between 80 and 84 then '80-84'
        when datediff(yy, b.dob, a.index_date) between 85 and 110 then '85+'
        else null
        end as age_cat
from
    st_rvo_cohort a
        join st_rvo_pat_all_unique b
            on a.patient_id = b.patient_id

Query started at 05:53:31 PM GMT Daylight Time; Query executed in 0.14 m

,status
0,Table NK_MND_PATIENT_DEMOGRAPHICS successfully...


## Master table

In [31]:
snow.drop_table("st_rvo_master_table")

DROP TABLE IF EXISTS sandbox_analytics.sandbox.nk_mnd_master_table;
Table sandbox.nk_mnd_master_table dropped!


In [32]:
%%read_sql

create or replace table st_rvo_master_table as

select
    a.patient_id,
    null as wt,
    a.gender,
    a.age,
    a.age_cat,
    b.rvo,
    b.other_ME
from
    st_rvo_patient_demographics a
        join st_rvo_comorbidites_agg b
            on a.patient_id = b.patient_id

Query started at 05:53:40 PM GMT Daylight Time; Query executed in 0.06 m

,status
0,Table NK_MND_MASTER_TABLE successfully created.


In [33]:
snow.select("select * from st_rvo_master_table limit 10")

,patient_id,wt,gender,age,age_cat,ftd,mnd
0,6BQ8xALqisW/BrRS,None,M,67,65-69,1,0
1,V0Nk8cY5HgnptYPJ,None,M,69,65-69,1,0
2,+0LZBQAPhbKxYFoa,None,F,66,65-69,1,0
3,4gK44p06gES2uTa7,None,M,68,65-69,1,0
4,VbU/gsiFk8gBekLF,None,M,74,70-74,1,0
5,tHBUOOK0EdyFw/JW,None,F,69,65-69,1,0
6,E/WaTaNUuI8/8gRb,None,F,66,65-69,1,0
7,DSYu9zO+1dWJk3Ae,None,F,75,75-79,1,0
8,CDXrBk4sQZ8bzAfl,None,M,71,70-74,1,0
9,eD/7NgasTXTcRotw,None,M,90,85+,1,0


# Summary results

In [34]:
%%read_sql

select
    --age_cat,
    sum(rvo) as rvo_pts,
    sum(other_ME) as ME_pts,
    round(sum(rvo)/sum(other_ME)*100, 2) as ME_perc
from
    st_rvo_master_table
--group by 1
--order by 1

Query started at 05:53:45 PM GMT Daylight Time; Query executed in 0.03 m

,ftd_pts,mnd,mnd_perc
0,30707,754,2.46


In [35]:
%%read_sql

select
    age_cat,
    sum(rvo) as rvo_pts,
    sum(other_ME) as ME_pts,
    round(sum(rvo)/sum(other_ME)*100, 2) as ME_perc
from
    st_rvo_master_table
group by 1
order by 1

Query started at 05:53:47 PM GMT Daylight Time; Query executed in 0.03 m

,age_cat,ftd_pts,mnd,mnd_perc
0,00-04,8,0,0.00
1,05-09,10,0,0.00
2,10-14,7,0,0.00
3,15-19,11,0,0.00
4,20-24,12,1,8.33
5,25-29,31,0,0.00
6,30-34,34,0,0.00
7,35-39,60,5,8.33
8,40-44,128,8,6.25
9,45-49,240,16,6.67
